## Import Modules

In [1]:
%pip -q install tensorflow==2.15.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.1 MB/s eta 0:00:00


In [3]:
import keras
from keras import layers

In [4]:
keras.__version__

'2.15.0'

In [5]:
%pip -q install pennylane==0.35.1 rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 44.8 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
import pennylane as qml
import rdkit
from rdkit import Chem, RDLogger
from rdkit.Chem import Crippen, QED
import warnings

In [7]:
import tensorflow as tf
tf.__version__

'2.15.1'

In [8]:
RDLogger.DisableLog("rdApp.*")
warnings.filterwarnings('ignore')

In [9]:
qml.about()

Name: PennyLane
Version: 0.35.1
Summary: PennyLane is a cross-platform Python library for quantum computing, quantum machine learning, and quantum chemistry. Train a quantum computer the same way as a neural network.
Home-page: https://github.com/PennyLaneAI/pennylane
Author: 
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: appdirs, autograd, autoray, cachetools, networkx, numpy, pennylane-lightning, requests, rustworkx, scipy, semantic-version, toml, typing-extensions
Required-by: PennyLane_Lightning

Platform info:           Linux-6.1.58+-x86_64-with-glibc2.35
Python version:          3.10.12
Numpy version:           1.25.2
Scipy version:           1.11.4
Installed devices:
- default.clifford (PennyLane-0.35.1)
- default.gaussian (PennyLane-0.35.1)
- default.mixed (PennyLane-0.35.1)
- default.qubit (PennyLane-0.35.1)
- default.qubit.autograd (PennyLane-0.35.1)
- default.qubit.jax (PennyLane-0.35.1)
- default.qubit.legacy (PennyLa

## Dataset

In [10]:
csv_path = keras.utils.get_file("qm9.csv", "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv")

29856825/29856825 [==============================] - 1s 0us/step


In [11]:
df = pd.read_csv(csv_path, usecols=["smiles", "h298", "u298", "g298"])
df = df.drop(labels=range(10000, 133885))

In [13]:
qed = pd.Series([QED.qed(Chem.MolFromSmiles(x)) for x in df["smiles"]], index=df.index, name="QED")
logP = pd.Series([Crippen.MolLogP(Chem.MolFromSmiles(x)) for x in df["smiles"]], index=df.index, name="LogP")

In [14]:
df = pd.concat([df, qed, logP], axis=1)

### Get the Adjacency Matrices

In [15]:
def smiles_to_graph(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    molecule = Chem.AddHs(molecule)
    adjacency = Chem.GetAdjacencyMatrix(molecule, useBO=True)
    return adjacency #no normalization, that comes later

In [16]:
vae_dataset = [smiles_to_graph(x) for x in df["smiles"]]
vae_dataset = [x for x in vae_dataset if x.shape[0] == 16]

In [17]:
train_dataset = vae_dataset[0 : int(0.8 * len(vae_dataset))]
val_dataset = vae_dataset[int(0.8 * len(vae_dataset)) : int(0.94 * len(vae_dataset))]
test_dataset = vae_dataset[int(0.94 * len(vae_dataset)) : ]

In [18]:
print(f"{len(train_dataset)} training examples.")

966 training examples.


In [19]:
qed_tensor = [df['QED'].iloc[i] for i in range(len(train_dataset))]

In [20]:
adjacency_tensor = np.array(train_dataset)
qed_tensor = np.array(qed_tensor)

In [21]:
latent_dim = 64
adj_shape = 16

## Implement the Model Components

### Implement the Quantum Graph Embedding

In [22]:
np.random.seed(1926)

In [67]:
def EquivarEmbed(qubits: int, layers: int = 1):
    gammas = np.random.randn(layers,)
    dev = qml.device("default.qubit.tf", wires=qubits)

    @qml.qnode(dev)
    def V8136(inputs, weights):
        for i in range(qubits): qml.Hadamard(wires=i)
        for layer in range(layers):
            for i in range(qubits):
                for j in range(i+1, qubits):
                    if inputs[i][j] != 0.:
                        qml.IsingZZ(gammas[layer] * inputs[i][j], wires=[i, j])

        qml.templates.AngleEmbedding(inputs, wires=range(qubits), rotation='X')
        qml.templates.BasicEntanglerLayers(weights, wires=range(qubits), rotation=qml.RX)
        return [qml.expval(qml.PauliX(i)) for i in range(qubits)]

    return qml.qnn.KerasLayer(V8136, weight_shapes={"weights": (layers, qubits)}, output_dim=16)

### Encoder

In [68]:
def Encoder(input_shape, qubits: int, L: int, **kwargs):
    adjacency = layers.Input(shape=input_shape)
    x = EquivarEmbed(qubits=16, layers=L)(adjacency)
    x = layers.Dense(units=2*latent_dim, activation="relu", name="dense")(x)
    x = layers.Dropout(rate=0.2, name="dropout")(x)
    z_mean = layers.Dense(units=latent_dim, activation="relu", name="z_mean")(x)
    log_var = layers.Dense(units=latent_dim, activation="relu", name="log_var")(x)
    return keras.Model(adjacency, [z_mean, log_var], name="encoder")

In [69]:
E = Encoder(input_shape=(adj_shape, adj_shape), qubits=16, L=1)

In [70]:
E.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_20 (InputLayer)       [(None, 16, 16)]             0         []                            
                                                                                                  
 keras_layer_13 (KerasLayer  (None, 16)                   16        ['input_20[0][0]']            
 )                                                                                                
                                                                                                  
 dense (Dense)               (None, 128)                  2176      ['keras_layer_13[0][0]']      
                                                                                                  
 dropout (Dropout)           (None, 128)                  0         ['dense[0][0]']         

### Decoder

In [72]:
def Decoder(input_shape):
    features = layers.Input(input_shape)
    x = layers.GlobalAveragePooling1D(data_format='channels_last', keepdims=False, name="pooling")(features)
    x = layers.Dense(units = adj_shape * adj_shape, activation="relu", name="expand")(x)
    x = layers.Dropout(rate=0.2, name="dropout")(x)
    x = layers.Reshape(target_shape=(adj_shape, adj_shape), name="reshape")(x)
    adjacency = layers.Softmax(axis=-1, name="softmax")(x)
    return keras.Model(features, adjacency, name="decoder")

In [73]:
D = Decoder(input_shape=(adj_shape, latent_dim))

In [74]:
D.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 16, 64)]          0         
                                                                 
 pooling (GlobalAveragePool  (None, 64)                0         
 ing1D)                                                          
                                                                 
 expand (Dense)              (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 reshape (Reshape)           (None, 16, 16)            0         
                                                                 
 softmax (Softmax)           (None, 16, 16)            0         
                                                           

### Sampling Layer

In [89]:
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_log_var)[1]
        dim = tf.shape(z_log_var)[2]
        epsilon = tf.keras.backend.random_normal(shape=(1, batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

## Build the Model

In [90]:
class QuantumVAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.property_prediction_layer = layers.Dense(1)
        self.train_total_loss_tracker = keras.metrics.Mean(name="train_total_loss")
        self.val_total_loss_tracker = keras.metrics.Mean(name="val_total_loss")

    def train_step(self, data):
        graph_real, qed_real = data[0]
        self.batch_size = tf.shape(qed_real)[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, qed_pred, graph_generated, = self(graph_real, training=True)
            total_loss = self._compute_loss(z_log_var, z_mean, qed_tensor, qed_pred, graph_real, graph_generated)
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.train_total_loss_tracker.update_state(total_loss)
        return {"loss": self.train_total_loss_tracker.result()}

    def _compute_loss(self, z_log_var, z_mean, qed_true, qed_pred, graph_real, graph_gen):
        adjacency_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.categorical_crossentropy(graph_real, graph_gen), axis=(1,),))
        kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), 1)
        kl_loss = tf.reduce_mean(kl_loss)
        property_loss = tf.reduce_mean(keras.losses.binary_crossentropy(qed_true, qed_pred))
        graph_loss = self._gradient_penalty(graph_real, graph_gen)
        return kl_loss + property_loss + graph_loss + adjacency_loss

    def _gradient_penalty(self, graph_real, graph_generated):
        alpha = tf.random.uniform([self.batch_size])
        alpha = tf.reshape(alpha, (self.batch_size, 1, 1))
        adjacency_interp = (graph_real * alpha) + (1 - alpha) * graph_generated
        with tf.GradientTape() as tape:
            tape.watch(adjacency_interp)
            _, _, logits, _, _ = self(adjacency_interp, training=True)

        grads = tape.gradient(logits, adjacency_interp)
        grads_adjacency_penalty = (1 - tf.norm(grads[0], axis=1)) ** 2
        return tf.reduce_mean(grads_adjacency_penalty, axis=(-1))

    def call(self, inputs):
        z_mean, log_var = self.encoder(inputs)
        z = Sampling()([z_mean, log_var])
        gen_adjacency = self.decoder(z)
        property_pred = self.property_prediction_layer(z_mean)
        return z_mean, log_var, property_pred, gen_adjacency

In [91]:
scheduler = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate = 1e-4,
    decay_steps = 100,
    alpha=0.0,
    name="CosineDecay",
    warmup_target=1e-3,
    warmup_steps=10,
)
optimizer = keras.optimizers.Lion(learning_rate=scheduler)

In [92]:
model = QuantumVAE(E, D)
model.compile(optimizer)

## Train Model

In [ ]:
model(np.random.randn(1, 16, 16)) #build model first

In [94]:
model.summary()

Model: "quantum_vae_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Functional)        [(None, 64),              18704     
                              (None, 64)]                        
                                                                 
 decoder (Functional)        (None, 16, 16)            16640     
                                                                 
 dense_2 (Dense)             multiple                  65        
                                                                 
Total params: 35413 (138.33 KB)
Trainable params: 35409 (138.32 KB)
Non-trainable params: 4 (16.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit([adjacency_tensor[:100], qed_tensor[:100]], epochs=10, verbose=1)

Epoch 1/10
